In [ ]:
'''
KAVE-AI
Predicting US Economic Recessions
CS 499 - Capstone Project

'''

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, roc_auc_score, average_precision_score, make_scorer, ConfusionMatrixDisplay


# Unemployment Rate (01/1985 - 10/2023) 'https://fred.stlouisfed.org/series/UNRATE'
unemp_dataset = pd.read_csv('UNRATE.csv')

# Consumer Price Index for All Urban Consumers: All Items (01/1985 - 10/2023) 'https://fred.stlouisfed.org/series/CPIAUCSL'
cpi_dataset = pd.read_csv('CPIAUCSL.csv')

# 10-Year Treasury Constant Maturity Minus 3-Month Treasury Constant Maturity Monthly Avg (01/1985 - 10/2023) https://fred.stlouisfed.org/series/T10Y3M#0
t_10y_3m_dataset = pd.read_csv('T10Y3M.csv')

# S&P 500 (01/1985 - 10/2023) https://finance.yahoo.com/quote/%5EGSPC/history?p=%5EGSPC
sp_500_daily_dataset = pd.read_csv('^GSPC.csv')
sp_500_daily_dataset.rename(columns={'Date': 'DATE'}, inplace=True)
sp_500_daily_dataset['DATE'] = pd.to_datetime(sp_500_daily_dataset['DATE'])
sp_500_daily_dataset.set_index('DATE', inplace=True)

# we need the mean adj close for each month
sp_500_monthly_dataset = sp_500_daily_dataset['Adj Close'].resample('M').mean()

start_date = '1985-01-02'
end_date = '2023-10-31'

sp_500_monthly_dataset = sp_500_monthly_dataset[start_date:end_date]
sp_500_monthly_dataset = sp_500_monthly_dataset.reset_index()
sp_500_monthly_dataset['DATE'] = sp_500_monthly_dataset['DATE'].dt.to_period("M").dt.strftime("%m/%Y")

start_date = '01/1985'

sp_500_monthly_dataset = sp_500_monthly_dataset[sp_500_monthly_dataset['DATE'] >= start_date]


# save dataframe to a new CSV file, used for test/training
sp_500_monthly_dataset.to_csv('SP500.csv', index=False)


# Historical dataset (1985 - 2023) 'https://fred.stlouisfed.org/series/USREC#0'
historical_recession_dataset = pd.read_csv('USREC.csv')
historical_recession_dataset['DATE'] = pd.to_datetime(historical_recession_dataset['DATE'])
historical_recession_dataset['DATE'] = historical_recession_dataset['DATE'].dt.to_period("M").dt.strftime("%m/%Y")
historical_recession_dataset = historical_recession_dataset[historical_recession_dataset['DATE'] >= start_date]

# merge unemp + cpi
merged_initial_dataset = pd.merge(unemp_dataset, cpi_dataset, on='DATE')

# merge (unemp + cpi) + (t10y3m)
merged_second_dataset = pd.merge(merged_initial_dataset, t_10y_3m_dataset, on='DATE')
merged_second_dataset['DATE'] = pd.to_datetime(merged_second_dataset['DATE'])
merged_second_dataset['DATE'] = merged_second_dataset['DATE'].dt.to_period("M").dt.strftime("%m/%Y")
merged_second_dataset = merged_second_dataset[merged_second_dataset['DATE'] >= start_date]


# merge (all indicators) + (sp500)
merged_dataset = pd.merge(merged_second_dataset, sp_500_monthly_dataset, on="DATE")


# historical recession data + all indicators
merged_with_actual = pd.merge(merged_dataset, historical_recession_dataset, on='DATE', how='left', suffixes=('_model', '_actual'))

X = merged_dataset[['UNRATE', 'CPIAUCSL', 'T10Y3M', 'Adj Close']]
y_actual = merged_with_actual['USREC']
X = X.reset_index(drop=True)  # reset the index of X_test before splitting

# 80% of the data is used to train the model, 20% is used to test
X_train, X_test, y_train_actual, y_test_actual = train_test_split(X, y_actual, test_size=0.2, random_state=23)

# Standardization
scaler = StandardScaler()
X_scaled_train = scaler.fit_transform(X_train)
X_scaled_test = scaler.transform(X_test)
X_scaled_test = pd.DataFrame(X_scaled_test, index=X_test.index) # reset the index of X_scaled_test after scaling
X_test = X_test.reset_index(drop=True) # reset the index of X_test after scaling


# Training
class_weights = {0: 1, 1: 5.5}

svm_model = SVC(kernel='rbf', C = 100, class_weight=class_weights, gamma = 1) # implemented kernel trick
svm_model.fit(X_scaled_train, y_train_actual)

'''
# GridSearch Testing

scorer = make_scorer(precision_score, pos_label=0)
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1, 10],
    'class_weight': [class_weights]
}

grid_search = GridSearchCV(svm_model, param_grid, cv=5, scoring=scorer)
grid_search.fit(X_scaled_train, y_train_actual)
best_params = grid_search.best_params_

# Get the best model
print("Best Parameters:", best_params)
print("Best Hyperparameters:", grid_search.best_params_)

best_svm_model = grid_search.best_estimator_

print("Best SVM model", best_svm_model)
'''


# Make predictions on the scaled set
X_scaled = scaler.transform(X)
merged_dataset['Recession_Predicted_Test'] = svm_model.predict(X_scaled)


# Metrics for our model's predictions
accuracy_merged = accuracy_score(y_actual, merged_dataset['Recession_Predicted_Test'])
precision_merged = precision_score(y_actual, merged_dataset['Recession_Predicted_Test'], average = 'weighted')
recall_merged = recall_score(y_actual, merged_dataset['Recession_Predicted_Test'], average = 'weighted')
conf_matrix_merged = confusion_matrix(y_actual, merged_dataset['Recession_Predicted_Test'])
f1 = f1_score(y_actual, merged_dataset['Recession_Predicted_Test'], average = 'weighted')
roc_auc = roc_auc_score(y_actual, merged_dataset['Recession_Predicted_Test'])
average_precision = average_precision_score(y_actual, merged_dataset['Recession_Predicted_Test'], average = 'weighted')

print("Model's Prediction vs Actual Recessions:")
print(f"Accuracy: {accuracy_merged * 100:.2f}%")
print(f"Precision: {precision_merged * 100:.2f}%")
print(f"Recall: {recall_merged * 100:.2f}%")
print(f"Confusion Matrix:\n{conf_matrix_merged}") # how inaccurate is our model compared to real historical data
print(f"F1 Score: {f1 * 100:.2f}%")
print(f"ROC AUC Score: {roc_auc * 100:.2f}%")

Model's Prediction vs Actual Recessions:
Accuracy: 98.71%
Precision: 98.71%
Recall: 98.71%
Confusion Matrix:
[[427   3]
 [  3  33]]
F1 Score: 98.71%
ROC AUC Score: 95.48%
